In [ ]:
#!/home/xyang18/miniconda3/envs/pytorch/bin/ python
# -*- coding: utf-8 -*-
# Python version: 3.6

import os
import sys
import copy
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import WeightedRandomSampler, TensorDataset
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, BatchNorm1d, Dropout, Flatten, BCELoss
from torch.optim import Adam, SGD
# from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
gpu_id=3

In [ ]:
if gpu_id>=0:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    cuda_id = "cuda:" + str(0)  # cuda:2

device = torch.device(cuda_id if torch.cuda.is_available() else "cpu")
print("Device:", device)
if (torch.cuda.is_available()):
    torch.cuda.set_device(cuda_id)
    print("Current GPU ID:", torch.cuda.current_device())

In [ ]:
window_len = 512 # 512
stride_len = 20 # 100
act_list = [1, 2, 3, 4, 5, 6, 7, 12, 13, 16, 17, 24]
# act_list = [1, 2]

In [ ]:
# opts = get_args()
X=[]
user_labels=[]
act_labels=[]

# columns for IMU data
imu_locs = [4,5,6, 10,11,12, 13,14,15, 
            21,22,23, 27,28,29, 30,31,32, 
            38,39,40, 44,45,46, 47,48,49
           ] 

scaler = MinMaxScaler()
# scaler = StandardScaler()

for uid in np.arange(1,10):
    path = '../../../PAMAP2_Dataset/Protocol/subject10' + str(uid) + '.dat'
    df = pd.read_table(path, sep=' ', header=None)
    act_imu_filter = df.iloc[:, imu_locs] 

    for act_id in range(len(act_list)):
        act_filter =  act_imu_filter[df.iloc[:, 1] == act_list[act_id]]
        act_data = act_filter.to_numpy()
        if act_data.shape[0] > 0:      
            # scaler = StandardScaler()
            # scaler = MinMaxScaler()
            if uid==1 and act_list[act_id] == act_list[0]:
                scaler.fit(act_data)
            act_data = scaler.transform(act_data)
            
        act_data = np.transpose(act_data)

        # sliding window segmentation
        start_idx = 0
        while start_idx + window_len < act_data.shape[1]:
            window_data = act_data[:, start_idx:start_idx+window_len]
            downsamp_data = window_data[:, ::3] # downsample from 100hz to 33.3hz
            downsamp_data = np.nan_to_num(downsamp_data) # remove nan

            X.append(downsamp_data)
            user_labels.append(uid)
            act_labels.append(act_id)
            start_idx = start_idx + stride_len

In [ ]:
X = np.array(X).astype('float32')
X = X.reshape(X.shape[0], 1, X.shape[1], X.shape[2]) # convert list to numpy array
act_labels = np.array(act_labels).astype('float32')
act_labels = act_labels.reshape(act_labels.shape[0],1)
act_labels = to_categorical(act_labels, num_classes=len(act_list))

In [ ]:
class MLP(Module):   
    def __init__(self, num_classes):
        super(MLP, self).__init__()
        self.mlp_layers = Sequential(
            Linear(256, 512), # change the first dimension based on your data
            Dropout(0.5),
            Linear(512, 256),
            Dropout(0.5),
            Linear(256, 256),
            Linear(256, 128),
            Dropout(0.5),
            Linear(128, 16),
            Dropout(0.5),
            Linear(16, num_classes),
            torch.nn.Softmax(dim=1)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = torch.reshape(x, (x.shape[0], -1))
        x = self.mlp_layers(x)
        return x

In [ ]:
# kernel size, channel size, stride, paddings are hyper parameters and can be tuned
class CNN(Module):   
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 64, kernel_size=[5,2], stride=1, padding=1), # change the input channel based on your data
            # Conv2d(len(act_list), 64, kernel_size=[5,2], stride=1, padding=1), # change the input channel based on your data
            ReLU(inplace=True),
            BatchNorm2d(64),
            MaxPool2d(kernel_size=[1,2]),
            Dropout(0.25),
            # Defining another 2D convolution layer
            Conv2d(64, 64, kernel_size=[5,2], stride=1, padding=1),
            ReLU(inplace=True),
            BatchNorm2d(64),
            MaxPool2d(kernel_size=[1,2]),
            Dropout(0.25),

            Conv2d(64, 32, kernel_size=[5,2], stride=1, padding=1),
            ReLU(inplace=True),
            BatchNorm2d(32),
            MaxPool2d(kernel_size=[1,2]),
            Dropout(0.25),
            Flatten(),
            Linear(14784,128), # change the dimension based on your own data
            ReLU(inplace=True),
            BatchNorm1d(128),
            Dropout(0.5),
            Linear(128, 16),
            ReLU(inplace=True),
            BatchNorm1d(16),
            Dropout(0.5),
            Linear(16, num_classes),
            torch.nn.Softmax(dim=1)
        )

    # Defining the forward pass    
    def forward(self, x):
        # x = torch.reshape(x, (-1, 2, 128, 1)) # change to your data dimension, the example here is 2 channel x 128 samples/channel
        x = self.cnn_layers(x)
        return x

## Prepare your dataset:

In [ ]:
batch_size = 256

In [ ]:
## Prepare your dataset, split into x_train, y_train, x_test, y_test
print('Prepare data loaders...')

dataset = TensorDataset(torch.from_numpy(X), torch.from_numpy(act_labels))

# Train/Test dataset split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
trainDataset, testDataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(trainDataset,
    batch_size=batch_size, shuffle=True) 

test_loader = torch.utils.data.DataLoader(testDataset,
    batch_size=batch_size, shuffle=False)
print('Data loaders ready.')

In [ ]:
# tensor_train_x = torch.from_numpy(x_train.astype('float32')) # convert from numpy array to tensor
# tensor_train_y = torch.from_numpy(y_train.astype('float32')) # convert from numpy array to tensor

# training_set = TensorDataset(tensor_train_x, tensor_train_y)
# train_loader = list(torch.utils.data.DataLoader(training_set, batch_size=256, shuffle=True, pin_memory=True)) 

In [ ]:
model = CNN(len(act_list))
# model = MLP(your_num_classes)

model = model.to(device)

criterion = BCELoss()
# criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr=1e-3)
# optimizer = Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)
# optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9, nesterov=True)

In [ ]:
epochs = 20
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        # outputs = torch.reshape(outputs, [outputs.shape[0]])
        
        # print(torch.argmax(outputs, dim=1))
        loss = criterion(outputs, labels)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
# Save trained models
PATH = 'pamap2_cnn.pt'
torch.save(model.state_dict(), PATH)

## Evaluation on the test set

In [ ]:
# tensor_test_x = torch.from_numpy(x_test.astype('float32'))
# tensor_test_y = torch.from_numpy(y_test.astype('float32'))

# test_set = TensorDataset(tensor_test_x, tensor_test_y)
# test_loader = list(torch.utils.data.DataLoader(test_set, batch_size=512, shuffle=True, pin_memory=True)) 

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)        
        # calculate outputs by running images through the network
        outputs = model(images)
        # print(outputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        # print(predicted)
        # predicted = torch.argmax(outputs.data.cpu(), axis=1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f'Test Accuracy: {100 * correct // total} %')